In [ ]:
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel


model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(text=["a photo of a cat", "a photo of a dog"], return_tensors="pt", padding=True)
# outputs = model(**inputs)
# logits_per_image = outputs.logits_per_image # this is the image-text similarity score
# probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities


In [30]:
print(inputs["input_ids"])


tensor([[49406,   320,  1125,   539,   320,  2368, 49407],
        [49406,   320,  1125,   539,   320,  1929, 49407]])


In [ ]:
processor(text=["a photo of a cat", "a photo of a dog"], return_tensors="pt", padding=True, trucation=True)

In [9]:
print(a[0].shape)

torch.Size([3, 224, 224])


In [10]:
b = list(a)

In [11]:
print(b[0].shape)

torch.Size([3, 224, 224])


In [7]:
a["b"] = 2


In [8]:
a

OrderedDict([('a', 1), ('b', 2)])

In [4]:
from datasets import ClassLabel
c = ds["train"].info.features['label']

In [5]:
print(
    f"{c.int2str(0)}\n"
    f"{c.str2int('apple_pie')}\n"
    f"{c.num_classes}\n"
)

apple_pie
0
101



In [10]:
c.names[:5]

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare']

In [8]:
import numpy as np
a = np.array([[1, 2, 3], [4, 5, 6]])

print(
    a[[0, 1], [0, 1]]
)

[1 5]


In [2]:
from collections import OrderedDict
from queue import PriorityQueue


In [10]:
print(a[0].get())

(1, 2)


In [11]:
a[2] = PriorityQueue()

In [14]:
a[1] = PriorityQueue()

In [15]:
a.keys()

odict_keys([0, 2, 1])

In [18]:
import torch
def compute_cache_logits(image_features, cache, alpha, beta, clip_weights, neg_mask_thresholds=None):
    """Compute logits using positive/negative cache."""
    with torch.no_grad():
        cache_keys = []
        cache_values = []
        for class_index in sorted(cache.keys()):
            for item in cache[class_index]:
                cache_keys.append(item[0])
                if neg_mask_thresholds:
                    cache_values.append(item[2])
                else:
                    cache_values.append(class_index)

        cache_keys = torch.cat(cache_keys, dim=0).permute(1, 0)
        if neg_mask_thresholds:
            cache_values = torch.cat(cache_values, dim=0)
            cache_values = (((cache_values > neg_mask_thresholds[0]) & (cache_values < neg_mask_thresholds[1])).type(torch.int8)).cuda().half()
        else:
            cache_values = (F.one_hot(torch.Tensor(cache_values).to(torch.int64), num_classes=clip_weights.size(1))).cuda().half()

        affinity = image_features @ cache_keys
        cache_logits = ((-1) * (beta - beta * affinity)).exp() @ cache_values
        return alpha * cache_logits

In [1]:
a = PriorityQueue()
a.put(1)
a.put(2)
a.put(3)
a.put(-1)

NameError: name 'PriorityQueue' is not defined

In [27]:
for x in a.queue:
    print(x)


-1
1
3
2


In [28]:
a = [1, 2, 3]
print(
    a[:2] + [a[2]]
)

[1, 2, 3]


In [31]:
torch.softmax(torch.randn((1, 2)), dim=-1).shape

torch.Size([1, 2])

In [32]:
torch.tensor(1).shape

torch.Size([])

In [33]:
torch.argmax(torch.randn((1, 2)), dim=-1).shape

torch.Size([1])

In [34]:
a = [1, 2, 3]
b = [4, 5, 6]
c = [7, 8, 9]

for x in zip(a, b, c):
    print(x)

(1, 4, 7)
(2, 5, 8)
(3, 6, 9)


In [3]:
a = PriorityQueue(maxsize=1)

In [ ]:
a.put(1)
a.put(2)


In [4]:
class A:
    x: int
    y: int
    def __iter__(self):
        return iter([self.x, self.y])
    
a = A()

In [5]:
a.x = 1
a.y = 2

In [6]:
x, y = a

In [4]:
import torch
list((torch.randn(2, 2), torch.randn(2, 2)))

[tensor([[ 0.1403, -0.1369],
         [ 0.2447,  0.6363]]),
 tensor([[ 1.6144,  0.1036],
         [-1.6141, -0.2780]])]

In [7]:
list(torch.randn(2, 2).unbind(dim=0))[0]

tensor([-0.8343,  0.8018])

In [23]:
from datasets import Dataset

ds = Dataset.from_dict({"a": [1, 2, 3], "b": [4, 5, 6]})
fn = lambda x: {
        "a": [torch.randn((2, 2)) for _ in x["a"]],
        "b": x["b"]
    }

In [24]:
fn(ds[:2])

{'a': [tensor([[ 0.3739,  0.1242],
          [-0.3914, -0.8448]]),
  tensor([[-0.5382, -0.7050],
          [-0.8535,  1.9457]])],
 'b': [4, 5]}

In [ ]:
from datasets import load_dataset

ds = load_dataset("barkermrl/imagenet-a")

In [40]:
def convert_wordnet_to_cls(name: str):
    from nltk.corpus import wordnet
    return wordnet.synset_from_pos_and_offset(
        'n', int(re.search(r"^n(\d{8})$", name).group(1))
    ).name().split('.')[0]

a = ds["train"].info.features["label"].names
a = [convert_wordnet_to_cls(x) for x in a]

with open("imagenet-a.txt", "w", encoding='utf-8') as f:
    f.write(a)


DatasetDict({
    validation: Dataset({
        features: ['image', 'label', 'origin', 'english_label'],
        num_rows: 10980
    })
})

In [12]:
a.int2str(0)

'n01498041'

In [7]:
from nltk.corpus import wordnet

wordnet.synset_from_pos_and_offset('n', 1531178).name().split('.')[0]

'goldfinch'

In [18]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\17299\AppData\Roaming\nltk_data...


True

In [26]:
import re
p = re.search(r"^n(\d{8})$", "n01498041")


In [28]:
int(p.group(1))

1498041